In [1]:
search_for = 139
start_from = 0
threads = 10
thread = 4
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.0924382209777832
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 5
all cases: 376006121
type: [1, 1, 1, 1, 139] 139
cases of this type: 373301041
100000 10.729486453460295
200000 21.272295406205867
300000 21.238005197061845
400000 22.58974805007553
500000 20.287057025252516
600000 22.945109330012443
700000 21.342320716618328
800000 23.925740901807224
900000 18.282298534109295
1000000 22.357021859050267
1100000 19.161924123057773
1200000 25.4067080652954
1300000 26.343016418659943
1400000 4.353497832532333
1500000 19.717303796902666
1600000 20.68271581430106
1700000 22.97591376664664
1800000 20.34058043404801
1900000 18.972359125845976
2000000 25.015304958572006
2100000 23.586217344101282
2200000 24.115733741499994
2300000 32.16149110784496
2400000 35.73036604990438
2500000 37.36762698662626
2600000 35.7810987059235
2700000 18.095989961137256
2800000 25.976870730577932
2900000 48.3236115126699
3000000 46.7873

29700000 38.41400770307302
29800000 13.899084595475616
29900000 35.14676699969889
30000000 39.75549567380827
30100000 38.20140165742657
30200000 39.63638213966134
30300000 37.818754640518705
30400000 38.677406356179745
30500000 37.39554183505846
30600000 37.70147628086896
30700000 38.45389047854757
30800000 34.11623165405043
30900000 16.88343078269712
31000000 27.149268666400374
31100000 36.31043838625465
31200000 36.19065379479678
31300000 40.09298275744774
31400000 37.02828685056406
31500000 37.03403729737516
31600000 35.815415151078845
31700000 39.715228122430744
31800000 35.88870514984854
31900000 39.604514601825265
32000000 25.106286155991707
32100000 25.1028375467834
32200000 36.33359826338771
32300000 39.03456669749872
32400000 37.69953570182229
32500000 15.217413479637491
32600000 33.32409503566301
32700000 38.921110130733034
32800000 37.71526386303129
32900000 37.82709337059339
33000000 37.06390809938199
33100000 36.17131258160316
33200000 37.44803036031412
33300000 35.7306875

59800000 33.58772127625665
59900000 33.97448739635239
60000000 31.01630056591787
60100000 33.495337061484236
60200000 30.525569748561033
60300000 32.07378923729287
60400000 25.46150623914105
60500000 12.505646755958674
60600000 30.095516992793147
60700000 31.810621454793495
60800000 30.858242707127356
60900000 32.30158625018098
61000000 32.4659635748036
61100000 32.09786215209944
61200000 32.76958451524347
61300000 31.775582129769706
61400000 14.441429540722847
61500000 34.427167285260886
61600000 33.55010377583405
61700000 33.49949484212984
61800000 33.34398314819056
61900000 35.939851592542645
62000000 34.65648609722356
62100000 35.55663402555161
62200000 29.34111595989919
62300000 16.74728731220753
62400000 33.68009882827115
62500000 34.29174783394463
62600000 33.6978956759977
62700000 34.83206716439674
62800000 32.40897924574358
62900000 30.902956467733617
63000000 31.57396804734602
63100000 21.785257733922947
63200000 17.159795466517007
63300000 29.697598975289193
63400000 31.6086

89600000 28.960283710121306
89700000 27.95625043797752
89800000 28.178421346657316
89900000 28.104052588279707
90000000 9.62873646048284
90100000 25.693809147332193
90200000 28.092515180223725
90300000 27.99111786564468
90400000 27.500492931673183
90500000 27.38946180165444
90600000 27.821203171674203
90700000 11.281201882710857
90800000 29.071775895679906
90900000 30.294176950120843
91000000 29.699140341321453
91100000 30.864320134470038
91200000 30.181185278819953
91300000 31.3352863860328
91400000 31.829594365743812
91500000 30.817379255299176
91600000 29.33476026254056
91700000 31.047549662251615
91800000 29.10300029367753
91900000 30.097716889391354
92000000 13.472685653039404
92100000 22.757077745583732
92200000 27.41592956006612
92300000 27.610599768125972
92400000 25.64635861371756
92500000 26.397855123532658
92600000 27.828056844918198
92700000 4.618458116166992
92800000 27.92333169278168
92900000 26.851337359786253
93000000 26.39204337736308
93100000 27.023785955892652
932000

118700000 25.683635460640183
118800000 23.65620051717852
118900000 24.25787624431185
119000000 20.17058656111501
119100000 10.887201051084807
119200000 22.780249207666483
119300000 22.821658137925198
119400000 23.00329213811494
119500000 16.911233734240756
119600000 13.095359219032575
119700000 22.9394781391771
119800000 23.00944240510696
119900000 23.251403091631335
120000000 15.007890909802349
120100000 17.54518861959334
120200000 23.604526320235276
120300000 23.769137042377526
120400000 24.46430629954954
120500000 24.521926087792224
120600000 25.49708757905636
120700000 26.396994364309656
120800000 25.50535431933739
120900000 25.910639494523043
121000000 25.813486208392977
121100000 25.94826713537794
121200000 25.99049267701731
121300000 25.16610917051532
121400000 25.827298502428178
121500000 23.87489812369657
121600000 23.558642816937137
121700000 21.54458188892453
121800000 9.51880800344007
121900000 23.22526734738863
122000000 23.912154015059116
122100000 22.745335780001962
1222

...

233400000 14.344905528470639
233500000 14.365250693474728
233600000 14.806165164187359
233700000 13.394022194268812
233800000 14.370469313409654
233900000 13.05886930198959
234000000 14.71486997225555
234100000 12.6320229358391
234200000 13.404785495157
234300000 12.29344812339003
234400000 12.95637948947095
234500000 12.269080716228613
234600000 12.884768978681299
234700000 4.630732587929304
234800000 13.073277322570425
234900000 12.013946874624422
235000000 7.039336340297913
235100000 9.195293399037313
235200000 13.390611122014155
235300000 11.963027545729515
235400000 4.228946858187667
235500000 12.866558171816076
235600000 12.748954718883311
235700000 12.89477185999242
235800000 12.58356015738822
235900000 13.271715461579957
236000000 13.00094141399284
236100000 14.52682248657624
236200000 13.92851081749089
236300000 14.30983621158092
236400000 13.072200281199532
236500000 14.262130090519502
236600000 12.868246677291255
236700000 14.163026128800537
236800000 12.480301207287532
2369

262000000 11.539691615857164
262100000 10.053223237422122
262200000 11.22338676949297
262300000 10.059747629004695
262400000 6.351914056238199
262500000 7.760903298395765
262600000 11.334458673105436
262700000 10.764905445681292
262800000 11.892225438119064
262900000 11.755812099847407
263000000 11.539517957885982
263100000 11.088250712168524
263200000 12.14037622903039
263300000 10.92050292091283
263400000 11.729796664072447
263500000 11.01196178786024
263600000 11.456152247635
263700000 10.748018472914232
263800000 10.826252266628764
263900000 10.182277591125079
264000000 5.407545888457431
264100000 9.452038715300885
264200000 10.044243350070584
264300000 10.571043819488793
264400000 9.688774761939724
264500000 9.413701075860082
264600000 3.3262779775338074
264700000 10.900407116930566
264800000 9.783889872994063
264900000 10.695141866418599
265000000 9.727472747754835
265100000 6.321758111603529
265200000 7.1486752333900645
265300000 11.46217098606928
265400000 10.447096468234777
26

291000000 8.380014156556621
291100000 7.43190578816065
291200000 8.427237327843999
291300000 7.372881063434783
291400000 4.344961380569262
291500000 6.1110579712235635
291600000 8.136146735182267
291700000 7.648404382240743
291800000 7.880335288760187
291900000 7.588607546606469
292000000 8.544635736686299
292100000 8.22442779572226
292200000 2.4654773670680084
292300000 8.833969266132371
292400000 8.8512624676672
292500000 8.97325508412591
292600000 9.149619073037908
292700000 9.478094246850633
292800000 8.603244929344779
292900000 9.614320283457577
293000000 8.83055158502548
293100000 9.339238723899955
293200000 8.468696982288192
293300000 6.0384112712668125
293400000 5.642565289086577
293500000 9.117736644894071
293600000 7.6691472281513215
293700000 8.799335260707215
293800000 7.344077787702889
293900000 8.564364627827624
294000000 7.603914117766086
294100000 3.3610881238877472
294200000 6.8652275362207
294300000 8.41017575854655
294400000 7.938579919025253
294500000 8.597637132824

320300000 5.7843598803587435
320400000 5.7970279296125495
320500000 5.576356847978702
320600000 5.778094575773419
320700000 5.323516047200889
320800000 5.346517131679172
320900000 4.712257847752434
321000000 1.8019721218406122
321100000 5.5258817732536
321200000 5.217901414115792
321300000 5.793281779503203
321400000 5.731722161499516
321500000 6.177526315183242
321600000 5.563026295983382
321700000 6.304937088088693
321800000 5.625950002761534
321900000 4.087501504602124
322000000 3.8151320053912343
322100000 6.311986914750999
322200000 5.422854241319164
322300000 6.205741968834751
322400000 5.358946739913514
322500000 5.802224127074434
322600000 4.805257017964142
322700000 2.3311015244218725
322800000 5.386210643140675
322900000 5.892870330747595
323000000 5.378206391972781
323100000 5.488520764362871
323200000 5.19515620544182
323300000 5.359646470198817
323400000 4.956083359067678
323500000 4.976349850816583
323600000 3.7753881965546574
323700000 2.452023284929182
323800000 5.04143

349300000 1.7758307672422844
349400000 1.9419610962465426
349500000 2.8295615466487343
349600000 2.723847609953417
349700000 2.8750231369998813
349800000 2.7316179686858533
349900000 2.712311467379996
350000000 2.63922721253736
350100000 2.8293807193761373
350200000 2.60993498773995
350300000 2.7381629376426915
350400000 2.411819173756068
350500000 0.9248746386271781
350600000 2.151310091076791
350700000 2.7671780537204014
350800000 2.508572919641275
350900000 2.8145725908194006
351000000 2.497120184647382
351100000 2.6430020050533343
351200000 2.5677813623042502
351300000 2.7639027688899684
351400000 2.528150928400258
351500000 2.7385924725891693
351600000 2.7377259539659824
351700000 0.5896351473308578
351800000 2.7850049397802237
351900000 2.5375601895972806
352000000 0.8870511825407909
352100000 2.4655232510466933
352200000 2.5754769750912105
352300000 2.433370332225202
352400000 2.560288361985882
352500000 2.4754797371967077
352600000 2.42803614515746
352700000 2.4208370512349666
